In [ ]:
import redis
import json

# Define connection variables
host = 'redis-13857.c14.us-east-1-2.ec2.redns.redis-cloud.com'
port = 13857
password = 'okNaqK7r5dKE6stSis0FlUuiDlhH2t7M'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

# Efficient Returns Management

## Description of the use case
This use case involves managing product returns efficiently by tracking the status of returns, reasons for returns, and processing refunds or exchanges. It ensures a smooth returns process for customers, improving their overall satisfaction.

## Redis Data Structures Used
- **Hashes**: To store return information and statuses.
- **Streams**: To capture and process real-time return updates.
- **Sorted Sets**: To prioritize and manage returns based on urgency or value.

In [ ]:
# Mock data for the use case
mock_data = {
    'return_1001': {'order_id': 'order_1001', 'customer_id': 'cust_1', 'status': 'initiated', 'reason': 'defective item'},
    'return_1002': {'order_id': 'order_1002', 'customer_id': 'cust_2', 'status': 'received', 'reason': 'wrong item'},
    'return_1003': {'order_id': 'order_1003', 'customer_id': 'cust_3', 'status': 'processed', 'reason': 'changed mind'},
}
# Use Hashes to store return information and statuses
for return_id, return_info in mock_data.items():
    r.hset(return_id, mapping=return_info)
    
    
# Print the updated return information
def display_data(mock_data):
    for return_id in mock_data.keys():
        print(f"{return_id}: {r.hgetall(return_id)}")
        
print("Data before returns processing")
display_data(mock_data=mock_data)


# Use Streams to capture real-time return updates
r.xadd('return_updates', {'return_id': 'return_1001', 'status': 'received'})
r.xadd('return_updates', {'return_id': 'return_1002', 'status': 'processed'})

# Function to process return updates
def process_return_updates():
    stream_entries = r.xrange('return_updates')
    for entry in stream_entries:
        return_id = entry[1]['return_id']
        new_status = entry[1]['status']
        r.hset(return_id, 'status', new_status)

# Process the return updates
process_return_updates()

# Use Sorted Sets to prioritize and manage returns based on urgency
for return_id in mock_data.keys():
    status = r.hget(return_id, 'status')
    priority = 1 if status == 'initiated' else 2 if status == 'received' else 3  # Example priority
    r.zadd('return_priority', {return_id: priority})

print("Data after returns processing")
display_data(mock_data=mock_data)

# Print the prioritized returns
prioritized_returns = r.zrange('return_priority', 0, -1, withscores=True)
print('Prioritized Returns:')
for return_id, priority in prioritized_returns:
    print(f"Return order {return_id} has the priority of {priority}")